In [2]:
import ssl
# Bypass SSL verification
ssl._create_default_https_context = ssl._create_unverified_context
import os
os.environ['CURL_CA_BUNDLE'] = ''

import weaviate
import numpy as np
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Connect to Weaviate
client = weaviate.Client("http://weaviate:8080")

# Load Sentence Transformer Model for Embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Function to insert a question
def insert_question(question_text):
    embedding = model.encode([question_text]).tolist()[0]
    question_object = {
        "text": question_text,
        "embedding": embedding
    }
    client.data_object.create(question_object, "Question")

# Function to query for similar questions
def fetch_similar_questions(question_text, threshold=0.8):
    input_embedding = model.encode([question_text])
    
    query = client.query.get("Question", ["text", "embedding"]).do()
    questions = query['data']['Get']['Question']
    
    similar_questions = []
    for question in questions:
        question_embedding = np.array(question['embedding']).reshape(1, -1)
        similarity = cosine_similarity(input_embedding, question_embedding)[0][0]
        if similarity >= threshold:
            similar_questions.append(question['text'])
    
    return similar_questions

# Insert and fetch questions
insert_question("What is your favorite product?")
print(fetch_similar_questions("What's your favorite product and why?"))


['What is your favorite product?', 'What is your favorite product?']
